<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [9]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

In [16]:
# load data
path = './data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv'

amazon = pd.read_csv(path)

In [6]:
amazon.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


# 1. Text preprocessing

In [14]:
# extend stopwords to include extremely common words specific to the works
STOPWORDS = set(STOPWORDS).union(set(['amazon']))

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [26]:
def create_df(df):
    '''
    tokenize and keep only key columns
    '''
    df2 = df.copy()
    df2['tokens'] = df2['reviews.text'].apply(tokenize)
    df2['brand'] = df2['brand'].str.lower()
    df2 = df2[['name', 'brand', 'primaryCategories',
               'reviews.rating', 'reviews.text', 'tokens']]
    return df2

amazon = create_df(amazon)
amazon.head()    

,name,brand,primaryCategories,reviews.rating,reviews.text,tokens
0,AmazonBasics AAA Performance Alkaline Batterie...,amazonbasics,Health & Beauty,3,I order 3 of them and one of the item is bad q...,"[order, item, bad, quality, missing, backup, s..."
1,AmazonBasics AAA Performance Alkaline Batterie...,amazonbasics,Health & Beauty,4,Bulk is always the less expensive way to go fo...,"[bulk, expensive, way, products, like]"
2,AmazonBasics AAA Performance Alkaline Batterie...,amazonbasics,Health & Beauty,5,Well they are not Duracell but for the price i...,"[duracell, price, happy]"
3,AmazonBasics AAA Performance Alkaline Batterie...,amazonbasics,Health & Beauty,5,Seem to work as well as name brand batteries a...,"[work, brand, batteries, better, price]"
4,AmazonBasics AAA Performance Alkaline Batterie...,amazonbasics,Health & Beauty,5,These batteries are very long lasting the pric...,"[batteries, long, lasting, price, great]"


In [30]:
amazon['reviews.rating'].value_counts().sum

<bound method Series.sum of 5    19897
4     5648
3     1206
1      965
2      616
Name: reviews.rating, dtype: int64>

In [24]:
amazon.shape

(28332, 6)

# 2. Fit a Gensim LDA topic model

In [ ]:
def doc_stream(path):
    # read all files in path
    for f in os.listdir(path):
        # ignore subdirectory
        if os.path.isdir(f) == False:
            # if ends in .txt
            if f[-3:] == 'txt':
                # open the text file
                with open(os.path.join(path,f)) as t:
                    # read the text file
                    text = t.read().strip('\n')
                    # extract the tokens
                    tokens = tokenize(str(text))
                yield tokens

In [70]:
#**** what exactly does the generator function do??

def get_reviews(df, token_col):
    """
    Generator function that takes a dataframe with a tokenized column and yields a tokenized 
    row for processing
    
    Parameters:
    -----------
    df: dataframe
    taken_col: name of the column of tokenized text
    
    Yields:
    -------
    review_token: one record of tokenized review
    """
    assert token_col in df.columns, f"{token_col} does not exist!"
    
    for index, row in df.iterrows():
        review_token = row[token_col]
        yield review_token


In [71]:
# A Dictionary Representation of all the words in our corpus
# id2word = corpora.Dictionary(amazon['tokens'])
id2word = corpora.Dictionary(get_reviews(amazon, 'tokens'))

In [33]:
# test
# check to see if specific words appear and get the id number
id2word.token2id['broken']

4962

In [36]:
# test
amazon.loc[50, 'reviews.text']

'I definitely love the price and quantity.. My kids go tthrough them to fast. At least these will last a while..'

In [38]:
# test
id2word.doc2bow(tokenize("I definitely love the price and quantity.. My kids go tthrough them to fast. At least these will last a while.."))


[(18, 1), (92, 1), (163, 1), (201, 1), (259, 1), (260, 1), (261, 1)]

In [40]:
# our vocab across all these segments:

len(id2word.keys())

9621

In [42]:
# Let's remove extreme values from the dataset
# no_below: at least x occurences 
# no_above: cut words that appear beyond .85 %
id2word.filter_extremes(no_below=5, no_above=0.95)

In [44]:
len(id2word.keys())

3581

In [47]:
# a bag of words(bow) representation of our corpus
# Note: we haven't actually read any text into memory here
# although abstracted away, tokenization is happening in the doc_stream f(x)
corpus = [id2word.doc2bow(text) for text in amazon['tokens']]

In [49]:
# look at first corpus object, and the first ten tuples
corpus[0][:10]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1)]

In [51]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=8
                  )

In [52]:
# each topic has a different distribution of the same "stock" of words
lda.print_topics()

[(0,
  '0.055*"kindle" + 0.044*"great" + 0.043*"love" + 0.027*"screen" + 0.020*"use" + 0.017*"tablet" + 0.016*"easy" + 0.016*"new" + 0.015*"better" + 0.014*"reading"'),
 (1,
  '0.052*"perfect" + 0.041*"great" + 0.031*"tablet" + 0.024*"love" + 0.017*"kids" + 0.016*"needs" + 0.015*"games" + 0.015*"uses" + 0.013*"happy" + 0.013*"need"'),
 (2,
  '0.048*"time" + 0.045*"battery" + 0.029*"life" + 0.026*"good" + 0.026*"long" + 0.016*"expected" + 0.016*"batteries" + 0.015*"work" + 0.012*"quality" + 0.012*"great"'),
 (3,
  '0.166*"good" + 0.096*"great" + 0.066*"price" + 0.042*"value" + 0.040*"product" + 0.039*"tablet" + 0.025*"quality" + 0.025*"works" + 0.016*"batteries" + 0.015*"excellent"'),
 (4,
  '0.130*"great" + 0.102*"price" + 0.040*"batteries" + 0.026*"buy" + 0.025*"long" + 0.023*"deal" + 0.019*"best" + 0.019*"work" + 0.018*"beat" + 0.013*"bought"'),
 (5,
  '0.055*"recommend" + 0.038*"great" + 0.025*"tablet" + 0.022*"definitely" + 0.022*"highly" + 0.016*"brands" + 0.015*"price" + 0.015*"j

In [54]:
# use regex to remove the non-text components
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [55]:
topics = [' '.join(t[0:5]) for t in words]

In [56]:
# print out nicely 
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
kindle great love screen use

------ Topic 1 ------
perfect great tablet love kids

------ Topic 2 ------
time battery life good long

------ Topic 3 ------
good great price value product

------ Topic 4 ------
great price batteries buy long

------ Topic 5 ------
recommend great tablet definitely highly

------ Topic 6 ------
easy use like great tap

------ Topic 7 ------
batteries brand long work battery

------ Topic 8 ------
use batteries work great charge

------ Topic 9 ------
games fast play great tablet

------ Topic 10 ------
tablet kids great nice use

------ Topic 11 ------
tablet apps store google like

------ Topic 12 ------
loves bought old year tablet

------ Topic 13 ------
tablet case parental great use

------ Topic 14 ------
kindle read books tablet reading



# 3. Interpret - pyLDAvis

In [58]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [59]:
# evaluate the model
pyLDAvis.gensim.prepare(lda, corpus, id2word)

//anaconda3/envs/U4-S1-NLP-DS6/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.247526 -0.104529       1        1  15.685812
12     0.206118 -0.015390       2        1   8.818958
14     0.096326 -0.034512       3        1   8.353828
0      0.077568 -0.011379       4        1   7.770946
11     0.025175 -0.023394       5        1   7.366218
10     0.105714  0.051520       6        1   7.049659
13     0.123539 -0.020193       7        1   6.282538
4     -0.170312  0.074796       8        1   6.212766
3     -0.130227  0.209338       9        1   6.135597
6      0.083329 -0.102139      10        1   6.059352
2     -0.136533 -0.054208      11        1   4.956732
8     -0.119745 -0.166054      12        1   4.509349
1      0.121538  0.019840      13        1   3.943630
9      0.030763  0.098923      14        1   3.592925
5     -0.065726  0.077379      15        1   3.261683, topic_info=     Category         Freq          Term        Total  loglift  logprob
18    Default  7651.000000     batteries  7651.000000  30.0000  30.0000
33    Default  5420.000000          good  5420.000000  29.0000  29.0000
21    Default  8962.000000         great  8962.000000  28.0000  28.0000
17    Default  4872.000000         price  4872.000000  27.0000  27.0000
1856  Default  2157.000000         loves  2157.000000  26.0000  26.0000
2075  Default  2950.000000        kindle  2950.000000  25.0000  25.0000
2432  Default  6979.000000        tablet  6979.000000  24.0000  24.0000
754   Default  3088.000000          easy  3088.000000  23.0000  23.0000
651   Default  1785.000000         games  1785.000000  22.0000  22.0000
191   Default  2487.000000          kids  2487.000000  21.0000  21.0000
520   Default  2118.000000           old  2118.000000  20.0000  20.0000
47    Default  1148.000000       perfect  1148.000000  19.0000  19.0000
165   Default  1147.000000     recommend  1147.000000  18.0000  18.0000
386   Default   966.000000         value   966.000000  17.0000  17.0000
93    Default  4578.000000           use  4578.000000  16.0000  16.0000
104   Default  1762.000000          year  1762.000000  15.0000  15.0000
2     Default  2105.000000       battery  2105.000000  14.0000  14.0000
23    Default  2372.000000          long  2372.000000  13.0000  13.0000
172   Default  2045.000000          time  2045.000000  12.0000  12.0000
27    Default  3167.000000        bought  3167.000000  11.0000  11.0000
89    Default  3536.000000          love  3536.000000  10.0000  10.0000
9     Default  2232.000000          work  2232.000000   9.0000   9.0000
164   Default  2044.000000       product  2044.000000   8.0000   8.0000
20    Default  1323.000000         brand  1323.000000   7.0000   7.0000
408   Default  1206.000000          play  1206.000000   6.0000   6.0000
1267  Default   951.000000      daughter   951.000000   5.0000   5.0000
243   Default   668.000000          fast   668.000000   4.0000   4.0000
884   Default  1083.000000          gift  1083.000000   3.0000   3.0000
58    Default  1019.000000          life  1019.000000   2.0000   2.0000
7     Default  1291.000000       quality  1291.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
1980  Topic15    27.235384        serves    51.448643   2.7869  -5.8961
2612  Topic15    15.648247       pockets    24.363567   2.9802  -6.4503
2539  Topic15    12.042011  satisfactory    17.170441   3.0681  -6.7122
1515  Topic15    44.697239       purpose   111.894531   2.5053  -5.4007
73    Topic15   159.710297        brands   831.109619   1.7735  -4.1273
1391  Topic15    22.072990       college    44.845558   2.7141  -6.1063
703   Topic15    51.288811          gets   177.695221   2.1803  -5.2632
1442  Topic15    53.553162        laptop   209.419418   2.0593  -5.2200
2412  Topic15    61.792206          slow   277.283325   1.9217  -5.0769
672   Topic15    71.533295          lots   362.147034   1.8010  -4.9305
225   Topic15  

# 4. Select number of topics

In [76]:
from gensim.models.coherencemodel import CoherenceModel

def compute_coherence_values(dictionary, corpus, df, token_col, limit, start=2, step=3, passes=5):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    path : path to input texts
    limit : Max num of topics
    passes: the number of times the entire lda model & coherence values are calculated

    Returns:
    -------
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    coherence_values = []
    
    tokens = list(get_reviews(df, token_col))
    
    for iter_ in range(passes):
        for num_topics in range(start, limit, step):
            stream = amazon['tokens']
            model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, workers=4)
            coherencemodel = CoherenceModel(model=model,dictionary=dictionary,corpus=corpus, coherence='u_mass')
            coherence_values.append({'pass': iter_, 
                                     'num_topics': num_topics, 
                                     'coherence_score': coherencemodel.get_coherence()
                                    })

    return coherence_values

In [ ]:
# Can take a long time to run.
coherence_values = compute_coherence_values(dictionary=id2word, 
                                                        corpus=corpus,
                                                        df=amazon,
                                                        token_col='tokens',
                                                        start=2, 
                                                        limit=10, 
                                                        step=1,
                                                        passes=5)

In [ ]:
topic_coherence = pd.DataFrame.from_records(coherence_values)

In [78]:
topic_coherence.head()

NameError: name 'topic_coherence' is not defined

In [79]:
import seaborn as sns

ax = sns.lineplot(x="num_topics", y="coherence_score", data=topic_coherence)

NameError: name 'topic_coherence' is not defined

# 5. TO DO Interpret - Overall Model / Documents 

In [61]:
distro = [lda[d] for d in corpus]

def update(doc):
        d_dist = {k:0 for k in range(0,15)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

In [ ]:
# append new distribution, set index to titles
df = pd.DataFrame.from_records(new_distro, index=titles)

# column names = topics from earlier
df.columns = topics

df['author'] = df.reset_index()['index'].apply(lambda x: x.split('_')[0]).tolist()

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling